# Boston Crime Analysis

## Goals
We're going to be working with a dataset of roughly 270,000 criminal incidents in Boston for three years (March 2012 - Mar 2015). Data is hosted by the <a href=https://data.cityofboston.gov/Public-Safety/Crime-Incident-Reports/7cdf-6fgx>City of Boston</a> and released by the Boston Police Department.

Our goal is to construct a spatiotemporal model of crime rate in Boston, using a model from Professor Dawn Woodward's <a href= https://people.orie.cornell.edu/woodard/ZhouMattWood13.pdf>2015 paper modeling ambulance dispatch calls</a>. We'll construct a nonhomogenous point process model, using Gaussian mixture models to model the spatial distribution (Don't worry if this is foreign to you, we'll break everything down later).


## Outline

### Data Preprocessing
In this step, we'll show how to use command line tools to quickly preprocess your data.

### Exploring Temporal Dynamics
We'll dive deeper into the temporal dynamics of crime incidents, looking for weekly and daily patterns to determine how we should best construct our model.

### Exploring Spatial Dynamics
We'll look at a couple heat maps to check  that there are indeed time varying spatial components to Boston crime data. 

### Notes on Data Science
Why not just use something out of scikit learn or TensorFlow? Can't neural networks predict everything?

### Introduction to Poisson Point Processes and Gaussian Mixture Models
Stepping back for a bit from the dataset, we'll talk generally about the type of models we're going to be working with.

### Model Fitting
We'll construct a time varying Poisson point process to model crime rate 

### Conclusion
We'll use everything we've built to create a simple visualization tool for crime rate and support a function $crime(time, location)$ that outputs a single score measuring crime rate as a function of time and location.

## Data Processing
Let's take a look at our data set to see why this is necessary. The following section should be done in a Bash shell.

We can see that there's a lot of information - let's first examine the NatureCode and Incident_Type_Description fields, looking at all possible values these take on. 

In [1]:
import subprocess
subprocess.call("awk -F ',' '{print $2}' crimedata_nh.csv | sort | uniq -c | sort -nr > keys.csv", shell=True);
data = open('keys.csv').readlines()
print 'Distinct Codes: '+ str(len(data))
print ''.join(data)

Distinct Codes: 320
15999 LARCRT
15932 ARREST
14204 IVPER 
14149 IVPER
12302 MVA
11395 LARCEN
8376 HITRUN
6432 IVMV
6406 IVPREM
6101 BE
6018 VANRPT
5688 BEMVRT
5453 LSTPRP
5106 INVEST
4891 BERPT 
4468 PROP
4275 VAND
3945 THREAT
3906 IVMV  
3805 IVDRUG
3257 AB
3202 MISPER
2922 TSTOP 
2915 MVAINJ
2789 MVA   
2708 DISTRB
2675 MVANO 
2670 LARCIP
2495 ABIP  
2487 FDPROP
2442 ILLPRK
2298 STOLMV
2255 ABRPT 
2017 FIGHT 
1891 EDP2  
1856 UNCONS
1782 MVPED1
1665 TS
1665 NULL
1651 UNK   
1584 BEIP  
1565 HARPT 
1429 REQP  
1403 SSTOP 
1346 ARMROB
1327 INJOFF
1296 EDP
1291 MISC  
1192 CD11  
1162 SHOTS 
1162 FIGHT
1128 PKNIFE
1110 VIORDR
1101 PERGUN
1052 REQP
1024 ROBIP 
 979 CD35  
 944 CARST 
 930 MVBLDR
 894 FIREB 
 890 SS
 884 ALARM
 863 SHPLFT
 862 ROBBER
 860 LANTEN
 843 VANIP 
 762 NIDV
 704 MVPED2
 686 UNKEMS
 680 MISSIN
 674 MISC
 672 SHOTS
 659 MSNCOM
 647 ADMRPT
 640 ROBRPT
 628 CD11
 613 REQEP 
 601 FDWEAP
 598 DVIP  
 581 NIDV  
 580 STAB  
 577 PDRPT 
 545 PARTY 
 541 REMOVE
 509 CD9

A lot of these codes are pretty cryptic and there are a bunch of them. Let's see if the other field describing the crime gives better information.

In [ ]:
import subprocess
#just change to the third field
subprocess.call("<INSERT AWK COMMAND HERE> crimedata_nh.csv | sort | uniq -c | sort -nr > keys.csv", shell=True);
print ''.join(open('keys.csv').readlines())

Some of these are pretty obvious. There's no easy way to decipher these, so dig through to see if there's anything related.

Can you figure out what Aircraft is referring to using the grep utility?

Here's an example below.


Here's what I've figured out so far:
<ul>
<li> <b> Aircraft </b>: oddly enough, all the exact same location, even over the span of three years. My guess is it's not an actual crime, but something that gets reported regardless. </li>
<li> <b> InvProp, InvPer, InvVeh </b>: answered by Boston PD on their <a href=https://twitter.com/bostonpolice/status/598911393425956865>twitter</a> and guessed the rest. Investigating property, investigating person, investigating vehicle </li>
<li> <b> Val </b>: answered by Boston PD on their <a href=https://twitter.com/bostonpolice/status/598911393425956865>twitter</a> - violation of auto law (driving w/out proper registration/license) </li>



</ul>

Now that we've figured out what all of these mean, we can start doing basic analysis.
Let's write a file which will contain all keys, which we'll then modify to include only violent crimes.

In [ ]:
crimeValues = dict();
data = open('keys.csv').readlines()
for line in data:
    text = line.strip().split(' ');
    #lower all text
    descrip = ' '.join(text[1:]).lower(); count = int(text[0])
    crimeValues[descrip] = count;
# write to csv file
keys = sorted(crimeValues.keys())
f = open('crimeskey.csv', 'wb');
for key in keys:
    f.write(key +','+str(crimeValues[key])+',\n')
    


After renaming the file (to avoid accidentally overwriting our mapping), we can reload the file and assemble the dictionary.

In [ ]:
data = open('<CHANGE ME TO NEW FILE NAME>').readlines();
crimeMapping = dict();

for line in data:
    stuff = line.strip().split(',')
    crimeMapping[stuff[0]]= int(stuff[2])

#uncomment the block below to print out the dictionary
keys = sorted(crimeMapping.keys())
for key in keys:
    print key + ' '*(27 - len(key)) + str(crimeMapping[key])

Now let's process the raw data using our mapping, including only crimes we want to consider. This time, we'll create a final file called processed.csv to store all our actual data we want to look at, and to get rid of extraneous fields we no longer need. All we need are the descriptions, the timestamps, day of week, and location.

In [ ]:
wbfile = open('processed.csv', 'wb');
with open('crimedata_nh.csv') as file:
    for line in file:
        line = line.split(',');
        if (line[2].lower() in crimeMapping and crimeMapping[line[2].lower()] == 1):
            wline = [line[2].lower(), line[6], line[13], line[19][2:], line [20].strip()[:-3]]
            wbfile.write(','.join(wline)+'\n');
wbfile.close()
        

Great, now we've gotten preprocessing out of the way, we can finally start looking at our data for interesting patterns. I've written some general tools to help ourselves abstract data collection/parsing and the actual computation

In [ ]:
# converts a timestamp to a single time object (instead of Python's datetime)
class Time:
    def __init__(self, stamp):
        stuff = stamp.split(' ');
        date = stuff[0].split('/');
        time = stuff[1].split(':');
        period = stuff[2];
        toAdd = 12 if (period == 'PM') else 0;
        hh = int(time[0]);
        hh = 0 if (hh == 12) else int(time[0]);
        hh+=toAdd;
        mm = int(time[1]); sec = int(time[2]);
        mon = int(date[0]); dd = int(date[1]); yr = int(date[2]);
        self.yr = yr;
        self.dd = dd;
        self.mon = mon;
        self.hh = hh; 
        self.mm = mm;
        self.sec = sec;
        self.monthMap = dict([(0, 0), (1, 31), (2, 59), (3, 90), (4, 120), (5, 151), (6, 181),
                             (7, 212), (8, 243), (9, 273), (10, 304), (11, 334), (12, 365)])
    
    @property
    def secondsSinceDay(self):
        return (self.hh*60 + self.mm)*60+self.sec;
    
    @property #period where 2 hour windows
    def dayPeriod(self):
        return self.hh/2;
    
    @property
    def daysSinceYear(self):
        return self.monthMap[self.mon-1]+self.dd;

'''
Feed is a class that will iterate over the data. 
This will allow us to separate the logic for the details of reading a file/data and the actual computation.

### CALLBACK SIGNATURES ###
def beginCB(void):

def recordCB(data):
     data - dict with keys 'crime' (string), 'timestamp' (Time), 'dow' (int representing day of week)
         'lat' (float), 'long' (float)'

def endCB(void):
'''

class Feed:
    # take in a file name and lists different kinds of call backs
    # EX: Feed('processed.csv', [], [myfunc1, myfunc2], [myfunc3])
    def __init__(self, fname = 'processed.csv', beginCBs = [], recordCBs = [], endCBs = []):
        self._fname = fname;
        self._beginCBs = beginCBs;
        self._recordCBs = recordCBs;
        self._endCBs = endCBs
        self.daymap = dict([('Sunday', 0), ('Monday', 1), ('Tuesday', 2), ('Wednesday', 3), ('Thursday', 4), ('Friday', 5), ('Saturday', 6)])
    
    @property #read only file name once constructed.
    def source(self):
        return self._fname
    
    #run through files, calling appropriate callbacks
    def run(self):
        errCount = 0;
        for cb in self._beginCBs:
            cb();
        with open(self._fname) as file:
            for line in file:
                line = line.split(',');
                #Recall format is crime, timestamp, day, latitude, longitude
                data = dict();
                try:
                    data['lat'] = float(line[-1]);
                    data['long'] = float(line[-2]);
                    data['crime'] = line[0];
                    data['timestamp'] = Time(line[1]); data['dow'] = self.daymap[line[2]]
                    for cb in self._recordCBs:
                        cb(data);
                except:
                    errCount+=1;
        for cb in self._endCBs:
            cb();
        
        print 'Failed to read '+ str(errCount) + ' lines'
    
    # methods for adding callback functions to class
    def addRecordCB(self, cb):
        self._recordCBs.append(cb);
    
    def addBeginCB(self, cb):
        self._beginCBs.append(cb);

    def addEndCB(self, cb):
        self._endCBs.append(cb);       
        

We've written some code to structure our analysis. Let's see how to use it. 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

#This class will accumulate a list of times during a week 
class TimeSeriesCollect: 
    def __init__(self, crime):
        self.data = []
        self.prevperiod = 0;
        self.crime = crime
    
    def recordCB(self, data):
        pass
        #implement me! 
        #let's make our call back record the time of the crime during the week.
        
    ''' 
    Once we've collected the data, we get to actually do some interesting computation now.
    Let's bin into 10 minute windows to generate a time series, then plot this result.
    '''
    def endCB(self):
        ax = plt.axes([0, 1, 2, 0.5]);
        npdata = np.array(self.data);
        
        # manipulate the data so we have a vector of seconds
        
        # use the ax.hist function of pyplot to plot a distribution

        #Choose bins based on the width of windows minute windows - this is 6*24*7 bins (weekly)

        plt.title(self.crime);
        

Our model of crime data is going to use time as a variable, but at most we're going to take time of week into account (i.e. not differentiate between day of the year since we don't have that much data). But it's worth checking if we even need to go into this much level - does day of the week even impact when crimes occur significantly? Or can we get by just considering time of day. This would certainly simplify our analysis.

In [ ]:
# get a timeseries for crimes with code manslaug - how do we interface the two classes we've built?


Let's take a look at assault - it seems likely that weekend nights would have higher incidence rates.

In [ ]:
def plotTSeries(crime):
    pass
    # implement me - shouldn't be hard, just copy the code above

#I only looked at crimes with a large number of incidents
plotTSeries('drug charges');

In [ ]:
plotTSeries('all')

It looks like day of week does matter from a first glance, though it can be a little hard to tell when looking at all types of crime. I found <a href="http://www.itl.nist.gov/div898/handbook/pmc/section4/pmc4.htm">this resource</a> to be a good option for an introduction to time series analysis for someone with minimal statistics background.



In [ ]:
import math

#This class will accumulate a list of times during a week of an occurence of a crime.
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

#This class will accumulate a spatial distribution of a crime of times during a week 
class SpatialDistributionCollect: 
    def __init__(self, crime = 'all'):
        self.data = dict();
        self.crime = crime;
        for i in range(84):
            self.data[i] = [];
    
    def recordCB(self, data):
        pass
        #implement me!
        # first check if we should even be recording the crime
            '''if (data['lat'] < -2 and data['long'] > 2):'''
                #some data is incomplete and gets recorded with 0, 0 lat, long
                # let's ignore this information

    def plotAll(self):
        fig = plt.figure(figsize=(10, 10));
        ax = fig.add_subplot(1, 1, 1);
        
        # can you figure out how to plot all data at once?
        
    def endCB(self):
        fig = plt.figure(figsize=(5, 100));
        plt.title(self.crime);
        for i in range(20):
            self.data[i] = np.array(self.data[i])
            ax = fig.add_subplot(20, 1, i+1);
            
            #plot 20 periods data at once
            #implement me

We have some preliminary analysis that shows the strongest intraweekly variation occurs for aggravated and simple assault.

In [ ]:
def getSpatial(crime):
    collect = SpatialDistributionCollect(crime);
    f = Feed('processed.csv', [], [lambda data: collect.recordCB(data)], [lambda : collect.endCB()]);
    f.run()
    return collect;

In [ ]:
violentCrime = getSpatial('all');

It would actually appear that our initial assumption that crime rate's spatial distribution varies as a function of time is false. The spatial distributions look similar, but with varying intensity (or overall crime rate) as a function of time.

# Notes on Data Science
Let's back up for a little bit. We've now explored our data quite a bit, finding strong evidence of a temporal component to crime rate and some (though admittedly not much) evidence of a time varying spatial distribution.

Why am I walking you through a complicated model? Can't neural networks can learn everything??

It's worth taking some time to comment on the recent obsession with neural networks, in particular their application to finance.
<ul>
    <li> Viewing neural networks as a pseudo brain is a bad way to view them - what are they really doing? </li>
    <li> Are they really helpful for applications to quantiative finance and HFT? </li>
</ul>
   

Here are some tips for modeling data well:
<ul>
    <li> Always try and use something about the problem structure. What does this mean? </li>
    <li> What are some important features about this dataset we can use? What are some valid assumptions? </li>
    <li> Crime rates should vary smoothly with respect to time and space, Actual crime incidents are somewhat random, meaning the fact that a crime occurs at 11:11 pm does not necessarily mean it will occur in the future and we should model crime as stemming from an underlying process.</li>
</ul>

## Gaussian Mixture Models
Here we model a distribution by taking multiple Gaussians and adding them to get a simplified representation.
See the following link for a <a href= "https://www.ll.mit.edu/mission/cybersec/publications/publication-files/full_papers/0802_Reynolds_Biometrics-GMM.pdf">paper</a> with more detail.
This is what we're using to model the spatial distribution.


## Point Processes
### Poisson Point Processes
A generative model for occurrence of spatial events. 
We select a number of points according to a Poisson distribution and place each one uniformly at random.
(Why a Poisson distribution - exponential distribution for waiting time lends itself to a Poisson distribution).
### Inhomogeneous Point Process
This is what we're interested in - it allows us to capture the time varying spatial distribution generating the dataset.

Let's first aggregate all our data and fit it to a Gaussian mixture model. We'll identify the relevant Gaussian distributions in our model. First, let's take a sample to estimate how many components we think are relevant.

In [ ]:
violentCrime.plotAll()

In [ ]:
from scipy import linalg
from sklearn import mixture
from matplotlib.colors import LogNorm

# construct a Gaussian Mixture Model
def fitGMM(data, components, mincovar):
    # initialize number of components here
    
    #set up a gaussian mixture model object called gmm here
    # should only take two lines

    x = np.linspace(-71.20, -70.95)
    y = np.linspace(42.20, 42.45)
    X, Y = np.meshgrid(x, y)
    XX = np.array([X.ravel(), Y.ravel()]).T
    Z = -gmm.score_samples(XX)[0]
    Z = Z.reshape(X.shape)
    
    fig = plt.figure(figsize = (10, 10))
    ax = fig.add_subplot(111)

    CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0), levels=np.logspace(0, 3, 10))
    CB = plt.colorbar(CS, shrink=0.8, extend='both')
    
    means = gmm.means_
    
    #implement me here
    # plot both the rawdata 
    # and scatter plot the means of the GMM as well

    plt.title('Negative log-likelihood predicted by GMM')
    plt.axis('tight')
    plt.show()
    return gmm

In [ ]:
gmm1 = fitGMM(violentCrime.alldata, 14, 1e-3)

In [ ]:
gmm2 = fitGMM(violentCrime.alldata, 14, 1e-6)

In [ ]:
gmm3 = fitGMM(violentCrime.alldata, 17, 1e-6)

In [ ]:
gmm4 = fitGMM(violentCrime.alldata, 21, 1e-6)

In [ ]:
gmm5 = fitGMM(violentCrime.alldata, 23, 1e-6)

In [ ]:
print 'Means:'
for i in range(len(gmm4.means_)):
    print str(gmm4.means_[i]) + '\t' + str(gmm4.covars_[i]) + '\t' + str(gmm4.weights_[i]) + '\n';

Great, so now we've settled on an initial spatial distribution for our model, how do we incorporate time varying demand?

First, we'll construct a spatial time series to determine rates $\lambda_i$ for Poisson distributions for each interval $ i \in [1, 84]$. To do this, we need a way of grouping the number of crimes by period to generate an approximate distribution for each interval. Fortunately, we are given the data stream in order, which means that the callback functions below will work.

In [ ]:
#This class will accumulate a spatial distribution of a crime of times during a week 
class PoissonDistributionCollect: 
    def __init__(self, crime = 'all'):
        self.data = dict();
        self.crime = crime;
        self.prevperiod = 0;
        self.currentcount = 0;
        for i in range(84):
            self.data[i] = [];
    
    def recordCB(self, data):
        # Implement me
        # again, first check if crime is valid
        # recall that we can compute the period within the week by looking at the hour 
        #   and the day of the week from the timestamp
        
        # How should prevperiod and current count be updated if the period stays the same? if it changes?
        pass
        
        

    def plotAll(self):
        fig = plt.figure(figsize=(6, 6));
        ax = fig.add_subplot(1, 1, 1);
        plt.axis([0, 83, 0, 80])
        for i in range(1, 84):
            ax.scatter(np.repeat(i, len(self.data[i])), np.array(self.data[i]), alpha=0.05, color='purple');

    def endCB(self):
        self.plotAll();

In [ ]:
def getRateDistributions(crime):
    collect = PoissonDistributionCollect(crime);
    f = Feed('processed.csv', [], [lambda data: collect.recordCB(data)], [lambda : collect.endCB()]);
    f.run()
    return collect;

In [ ]:
# print out the rate distributions for all crimes - or explore a few

It turns out we don't need to do anything complicated to determine the rates $\lambda$. We'll prove below that the maximum likelihood estimator of a Poisson distribution is in fact simply taking the average of all samples.

$$\mathbb{P}(x_1, ..., x_n | \lambda) = \prod_{i = 1}^n \frac{e^{-\lambda}\lambda^{x_i}}{x_i!} = \frac{e^{-n\lambda}\lambda^{\sum x_i}}{\prod_{j=1} x_j!}$$

$$ ln \mathbb{P} = -n\lambda + (\ln \lambda) \sum_{i=1}^n x_i $$
Can you complete the proof from here?

In [ ]:
class PoissonRateFitter:
    def __init__(self, datasrc):
        self.collected = datasrc
        self.rates = dict();
        
    def computerate(self):
        for i in range(0, 84):
            pass
            #implement the max likelihood estimator (note np.average function)


In [ ]:
# make a call to compute the crime rates
# call this PoissonRateFitter object crimerates

In [ ]:
def plotAll(datasrc, ratesrc):
        data = datasrc.data
        rates = ratesrc.rates
        fig = plt.figure(figsize=(6, 6));
        ax = fig.add_subplot(1, 1, 1);
        plt.axis([0, 83, 0, 40])
        #can you figure out how to plot all distributions and rates (code is very similar to the section a few cells above)

In [ ]:
# call plotAll here with the right variables

At this point, we've essentially completed our model. We have decoupled the temporal and spatial components and now have a method to predict crime rate at any point. Let's build a simple function $getCrimeRate(t, loc)$ that will return our measure of crime rate as a function of time as well as a visualization tool.

In [ ]:
from scipy import linalg
import matplotlib as mpl
import gmaps

class STCrimeModel:
    
    def __init__(self, gmm, crimerates, rawdata):
        self.gmm = gmm;
        self.means = gmm.means_
        self.covar = gmm.covars_
        self.weights = gmm.weights_
        self.crimerates = crimerates.rates;
        self.rawdata = rawdata.data;
        
    # this function isn't very interesting, just adds ellipses
    def plotPeriod(self, period):
        assert( period < 84 and period >=0);
        fig = plt.figure(figsize = (7, 7))
        ax = fig.add_subplot(111)
        for i in range(len(self.means)):
            v, w = linalg.eigh(self.covar[i])
            
            # Plot an ellipse to show the Gaussian component
            angle = np.arctan2(w[0][1], w[0][0])
            angle = 180 * angle / np.pi  # convert to degrees
            v *= 300 #scaling to make stuff visible
            alphav = min(0.5*self.weights[i]*self.crimerates[period], 1)
            ell = mpl.patches.Ellipse(self.means[i], v[0], v[1], 180 + angle, alpha=alphav)
            ell.set_clip_box(ax.bbox)
            ax.add_artist(ell)
        plt.axis([-71.20, -70.95, 42.20, 42.45]);
        rawdata = self.rawdata[period]
        ax.scatter(rawdata[:,0], rawdata[:,1], alpha=0.05)
    
    def getCrimeRate(self, hour, day, loc):
        #day is a string (use _daymap helper), hour is int in range [0, 23]
        # compute the period
        # return the rate predicted by the gmm and the crime rate
        pass

    def _daymap(self, day):
        daymap = dict([('Sunday', 0), ('Monday', 1), ('Tuesday', 2), ('Wednesday', 3), ('Thursday', 4), ('Friday', 5), ('Saturday', 6)])
        return daymap[day];
        
        

In [ ]:
finalmodel = STCrimeModel(gmm4, crimerates, violentCrime)

In [ ]:
finalmodel.plotPeriod(1);

In [ ]:
print finalmodel.getCrimeRate(2, 'Sunday', (-71.05, 42.30))

We've now built a model that not only has a clear intuitive meaning, but additionally supports fast queries for comparing crime rates at different locations and different times. 

### Here are a couple ideas for further extension:
<ul>
 <li> Use Markov Chain Monte Carlo methods to allow time varying spatial distributions - let the Gaussian mixture components stay constant, but allow the mixture weights to vary over time. What aspect of the problem does this reflect? </li>
 <li> Improve the visualization tool by incorporating the gmaps plug in to overlay models on Google Maps</li>
 <li> Impose smoothing constraints on crime rate - we treated each interval as a distinct distribution.
</ul>

### Suggested Topics/Classes for Data Science:
Basic Programming (CS2110, CS1110) <br>
Machine Learning (CS4786, CS4780, STSCI4740) <br>
Stochastic Processes (MATH4740, ORIE3510) <br>
Basic Statistics (STSCI2100, STSCI 3080) <br>
Scientific Computing (CS4250, CS4260, STSCI3520) <br>
Differential Equations (MATH4200, MATH4280) <br>
Time Series Analysis (STSCI4550)

<b> Please watch for an emailed survey about the crash course!!! </b> We'd like to keep improving our crash course in future semesters. Your feedback means a lot to us, and it helps us improve from semester to semester.